In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyspark.sql import SparkSession, Row
from datetime import datetime

In [2]:
path_prefix = '/home/joeyresuento/Projects/data_training/data_sets/ml-25m'
# movies = pd.read_csv(f'{path_prefix}/movies.csv')
# ratings = pd.read_csv(f'{path_prefix}/ratings.csv')

In [3]:
spark = SparkSession \
    .builder \
    .appName("MyApp") \
    .getOrCreate()

spark.sparkContext.setLogLevel('ERROR')

24/05/03 14:37:26 WARN Utils: Your hostname, aljresuento1 resolves to a loopback address: 127.0.1.1; using 192.168.254.174 instead (on interface enp1s0f0)
24/05/03 14:37:26 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/03 14:37:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
movies_df = spark.read.option("header", True).csv(f'{path_prefix}/movies.csv')
ratings_df = spark.read.option("header", True).csv(f'{path_prefix}/ratings.csv')

In [5]:
ratings_df.createOrReplaceTempView('ratings')
movies_df.createOrReplaceTempView('movies')
# spark.sql("SELECT COUNT(1) FROM movies").show()
# spark.sql("SELECT COUNT(1) FROM ratings").show()

In [7]:
# top100_highest_rated_movies = spark.sql("SELECT \
#    movieId, rating, COUNT(1) as count \
#    FROM ratings \
#    GROUP BY movieId, rating \
#    ORDER BY rating, count DESC LIMIT 100")

# top100_highest_rated_movie_ids = top100_highest_rated_movies \
#    .rdd.map(lambda r: int(r.movieId)) \
#    .collect()

In [8]:
movies_df.printSchema()
ratings_df.printSchema()

root
 |-- movieId: string (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)

root
 |-- userId: string (nullable = true)
 |-- movieId: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- timestamp: string (nullable = true)



In [9]:
movie_ratings_df = spark.sql("SELECT m.movieId, m.title, m.genres, r.userId, r.rating, r.timestamp \
FROM movies m INNER JOIN ratings r ON m.movieId = r.movieId")

In [26]:
# movie_ratings_df.take(10)[0]

In [37]:
def enrich_row(row):
    year = datetime.fromtimestamp(int(row.timestamp)).year
    return Row(
        movieId=int(row.movieId),
        title=row.title,
        genres=row.genres,
        userId=int(row.userId),
        rating=float(row.rating),
        timestamp=int(row.timestamp),
        year=year
    )

enriched_movie_ratings = movie_ratings_df.rdd.map(enrich_row)

In [40]:
enriched_movie_ratings_df = spark.createDataFrame(enriched_movie_ratings)
enriched_movie_ratings_df.createOrReplaceTempView('movies_ratings')
enriched_movie_ratings_df.printSchema()

root
 |-- movieId: long (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)
 |-- userId: long (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: long (nullable = true)
 |-- year: long (nullable = true)



In [47]:
ratings_count_by_year = enriched_movie_ratings_df.rdd \
    .map(lambda r: (r.year, 1)) \
    .reduceByKey(lambda a, b: a + b)

In [50]:
ratings_count_by_year_list = ratings_count_by_year.collect()

In [54]:
ratings_count_by_year_list

[(2016, 1756856),
 (2000, 1737245),
 (2017, 1690321),
 (2001, 1057762),
 (2002, 777868),
 (2018, 1310712),
 (2019, 1202383),
 (2003, 920780),
 (2004, 1047089),
 (2005, 1613232),
 (2006, 1039236),
 (2007, 930435),
 (2008, 1019216),
 (2009, 810134),
 (2010, 792767),
 (2011, 676488),
 (1995, 3),
 (1996, 1429280),
 (2012, 634829),
 (1997, 626849),
 (2013, 516094),
 (1998, 272153),
 (2014, 478141),
 (2015, 1604061),
 (1999, 1056161)]